In [19]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# Training parameters

In [20]:
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28
num_classes = 10
timesteps =28
num_hidden = 128

X = tf.placeholder("float", [None,timesteps,num_input])
Y = tf.placeholder("float", [None,num_classes])

In [21]:
# Define Weights
weights ={
    'out':tf.Variable(tf.random_normal([num_hidden,num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# LSTM Input shape
- Samples - This is the len(dataX), or the amount of data points you have. Or individual training examples

- Time steps - This is equivalent to the amount of time steps you run your recurrent neural network. If you want your network to have memory of 60 characters, this number should be 60.

- Features - this is the amount of features in every time step. If you are processing pictures, this is the amount of pixels.

In [22]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=True)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
    

 # Functions used 
 - Logits is a function which operates on the unscaled output of earlier layers and on a linear scale to understand the linear units. In Mathematics, Logits is a function that maps probabilities ( [0, 1] ) from real values R ( (-inf, inf) ) .
 - Softmax function turns logits [2.0, 1.0, 0.1] into probabilities [0.7, 0.2, 0.1], and the probabilities sum to 1.
 - cross entropy: 
 H(p,q) = p(x)*log(q(x))
 
 Where p(x) is the true probability of event x and q(x) is the predicted probability of event x.

There if input any two numbers for p(x) and q(x) are used such that

0<p(x)<1 AND 0<q(x)<1

In [23]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [24]:
# start Training
with tf.Session() as sess:
    
    sess.run(init)
    
    
    for step in range(1,training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape the data to get 28x28 elements
        batch_x = batch_x.reshape((batch_size,timesteps,num_input))
        #Run the optimization
        sess.run(train_op,feed_dict={X:batch_x, Y:batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})  
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 3.5185, Training Accuracy= 0.070
Step 200, Minibatch Loss= 2.1969, Training Accuracy= 0.273
Step 400, Minibatch Loss= 1.9854, Training Accuracy= 0.391
Step 600, Minibatch Loss= 1.8148, Training Accuracy= 0.398
Step 800, Minibatch Loss= 1.5902, Training Accuracy= 0.531
Step 1000, Minibatch Loss= 1.7106, Training Accuracy= 0.445
Step 1200, Minibatch Loss= 1.4515, Training Accuracy= 0.547
Step 1400, Minibatch Loss= 1.4485, Training Accuracy= 0.531
Step 1600, Minibatch Loss= 1.3677, Training Accuracy= 0.594
Step 1800, Minibatch Loss= 1.3501, Training Accuracy= 0.508
Step 2000, Minibatch Loss= 1.3537, Training Accuracy= 0.594
Step 2200, Minibatch Loss= 1.3762, Training Accuracy= 0.516
Step 2400, Minibatch Loss= 1.2363, Training Accuracy= 0.602
Step 2600, Minibatch Loss= 1.1887, Training Accuracy= 0.656
Step 2800, Minibatch Loss= 1.0488, Training Accuracy= 0.672
Step 3000, Minibatch Loss= 1.1559, Training Accuracy= 0.648
Step 3200, Minibatch Loss= 1.1616, Training Acc

Type help() for interactive help, or help(object) for help about object.